### 📖 Reference  
This notebook is based on:  
**Christof Paar and Jan Pelzl (2009). _Understanding Cryptography: A Textbook for Students and Practitioners_. Springer.**


### 📘 Definition 8.2.1 — Group  

A **group** is a set $G$ with a binary operation $\circ$ such that:  
1. **Closure:** $a \circ b \in G$ for all $a,b \in G$.  
2. **Associativity:** $a \circ (b \circ c) = (a \circ b) \circ c$ for all $a,b,c \in G$.  
3. **Identity:** There exists an element $1 \in G$, called the identity element  such that $a \circ 1 = 1 \circ a = a$ for all $a \in G$.  
4. **Inverse:** For each $a \in G$, there exists $a^{-1} \in G$, called the inverse of a, with $a \circ a^{-1} = 1$.  
5. If $a \circ b = b \circ a$ for all $a,b$, then $G$ is **abelian**.  

### 🔑 Intuition

- **Closure:** performing the group operation on any two group elements always yields another element.  
- **Associativity:** grouping of operations doesn’t change the result.  
- **Identity:** there exists an element that leaves everything unchanged when used in the operation.  
- **Inverses:** every element has a counterpart that brings you back to the identity.  
- **Abelian:** if the order of applying the operation doesn’t matter, the group is abelian.




### 📘 Theorem 8.2.1  

The set  

$$
\mathbb{Z}_n^* = \{\, i \in \{0,1,\dots,n-1\} \;\mid\; \gcd(i,n)=1 \,\}
$$  

forms an **abelian group** under multiplication modulo $n$.  
The identity element is $1$.  

---

### 🔑 Intuition  

The integers that are coprime to $n$ behave well under multiplication modulo $n$:  

- **Closure:** the product of any two is still in $ \mathbb{Z}_n^* $.  
- **Associativity:** inherited from integer multiplication.  
- **Identity:** $1$ acts as the neutral element.  
- **Inverses:** each element has an inverse (from the extended Euclidean algorithm).  
- **Commutativity:** multiplication modulo $n$ is commutative.  






In [ ]:
def euclid(a, b):
    if a == 0:
        return b
    return euclid(b % a, a)

class Group:
    def __init__(self, modulus):
        if modulus <= 1:
            raise ValueError("modulus must be > 1 for a non-empty multiplicative group.")
        self.modulus  = modulus
        self.elements = set(e for e in range(1, modulus) if euclid(modulus, e) == 1)

    def multiplicativeGroupOp(self, a, b):
        # guard: only operate on group elements
        if a not in self.elements or b not in self.elements:
            raise ValueError("Operands must be elements of the group.")
        return (a * b) % self.modulus

    def getCardinality(self):
        return len(self.elements)

    def checkClosureRule(self):
        for a in self.elements:
            for b in self.elements:
                if (a * b) % self.modulus not in self.elements:
                    return False
        return True

    def checkAssociativeRule(self):
        op = self.multiplicativeGroupOp
        for a in self.elements:
            for b in self.elements:
                for c in self.elements:
                    if op(a, op(b, c)) != op(op(a, b), c):
                        return False
        return True

    def checkNeutralElementRule(self):
        e = 1
        if e not in self.elements:
            return False
        for x in self.elements:
            if self.multiplicativeGroupOp(x, e) != x: return False
            if self.multiplicativeGroupOp(e, x) != x: return False
        return True

    def checkInverseRule(self):
        e = 1
        for a in self.elements:
            has_inv = False
            for y in self.elements:
                if self.multiplicativeGroupOp(a, y) == e and self.multiplicativeGroupOp(y, a) == e:
                    has_inv = True
                    break
            if not has_inv:
                return False
        return True

    def checkAbelianRule(self):
        op = self.multiplicativeGroupOp
        for a in self.elements:
            for b in self.elements:
                if op(a, b) != op(b, a):
                    return False
        return True


### 🔑 Why do Inverse and Identity matter in cryptography?

At the heart of cryptography is the idea of **doing math forward easily** but making it **painfully hard to reverse** unless you know a secret.

Here’s the magic:  
- The **identity element $1$** is the “reset button.”  
- An inverse is the unique partner that brings you back to 1: $a \cdot a^{-1} \equiv 1 \pmod n$.  
- If you own the private key 🔒, the math collapses back to some simple form in just a few steps, smooth and elegant.  
- But if you’re an attacker 🚫, finding that "inverse" is like brute-forcing a Rubik’s cube with billions of combinations.

That’s why inverses power modern crypto:  

- 🗝 **RSA**: public vs private keys are tied by modular inverses.  
- 🔐 **Diffie–Hellman**: hides inverses inside exponentiation tricks.  
- 🚫 Without inverses, decryption wouldn’t exist — every lock 🔒 would be one-way only.

👉 In short: **inverses + identity = the secret handshake** that makes decryption possible for the good guys, but impossibly hard for hackers.  


### 🧪 Let’s Experiment!

Time to put the definitions into action.  
We’ll construct $ \mathbb{Z}_n^* $, list its elements, and check whether the group rules are satisfied.  


In [ ]:
from IPython.display import display, Markdown

def show_group_properties(n):
    """Create multiplicative group Z*_n and display its properties."""
    G = Group(n)

    display(Markdown(f"### Properties of $\\mathbb{{Z}}_{{{n}}}^*$"))
    display(Markdown(f"- **Modulus:** {G.modulus}"))
    display(Markdown(f"- **Elements:** {sorted(G.elements)}"))
    display(Markdown(f"- **Order ($|\\mathbb{{Z}}_{{{n}}}^*|$):** {G.getCardinality()}"))

    checks = {
        "Closure Rule": G.checkClosureRule(),
        "Associative Rule": G.checkAssociativeRule(),
        "Identity Rule": G.checkNeutralElementRule(),
        "Inverse Rule": G.checkInverseRule(),
        "Abelian Rule": G.checkAbelianRule()
    }

    for name, result in checks.items():
        display(Markdown(f"- **{name}:** {result}"))

# Example
n = 20
show_group_properties(n)


### Properties of $\mathbb{Z}_{20}^*$

- **Modulus:** 20

- **Elements:** [1, 3, 7, 9, 11, 13, 17, 19]

- **Order ($|\mathbb{Z}_{20}^*|$):** 8

- **Closure Rule:** True

- **Associative Rule:** True

- **Identity Rule:** True

- **Inverse Rule:** True

- **Abelian Rule:** True

### 📝 Quiz 1: Simple Proof of Inverse Uniqueness

Show that the inverse of any element in a group is unique.  

<details>
<summary>✅ Click to reveal the proof</summary>

1. Suppose $a \in G$ and both $b$ and $c$ are inverses of $a$.  
2. That means:  
   $$ab = e \quad \text{and} \quad ac = e$$  
3. Multiply $(ab)$ by $c$ on the right:  
   $$(ab)c = ec$$  
4. Simplify each side using associativity and identity:  
   $$b(ac) = c$$  
5. Conclude that $b = c$.  

Therefore, the inverse of $a$ is unique. ✅

</details>


### 📝 Quiz 2: Even or Odd Cardinality of $\mathbb{Z}_n^*$

Question: Is the cardinality $|\mathbb{Z}_n^*| = \varphi(n)$ (Euler’s totient function) even or odd?

[ ] Always even  
[ ] Always odd  
[ ] Depends on $n$  

<details>
<summary>✅ Click to reveal the answer</summary>

$\varphi(n)$ is even for all $n > 2$.  
Exceptions: $|\mathbb{Z}_1^*| = 1$ and $|\mathbb{Z}_2^*| = 1$, which are odd.

</details>


In [ ]:
# @title

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle, FancyArrowPatch
import ipywidgets as widgets
from IPython.display import display, clear_output, Math

def zstar(n):
    return [a for a in range(1, n) if np.gcd(a, n) == 1]

def inv_mod(a, n):
    t, nt = 0, 1
    r, nr = n, a
    while nr:
        q = r // nr
        t, nt = nt, t - q*nt
        r, nr = nr, r - q*nr
    if r != 1:
        raise ValueError(f"{a} has no inverse mod {n}")
    return t % n

def draw_smooth_self_loop(ax, x, y, *, offset=0.26, r_loop=0.18,
                          color="#b22222", loop_lw=1.8, head_size=12):
    R = np.hypot(x, y)
    ux, uy = x / R, y / R
    cx, cy = x + offset * ux, y + offset * uy
    loop = Circle((cx, cy), r_loop, fill=False, lw=loop_lw, color=color)
    ax.add_patch(loop)
    theta_tip = np.arctan2(y - cy, x - cx) + 0.9
    tip  = (cx + r_loop*np.cos(theta_tip),     cy + r_loop*np.sin(theta_tip))
    near = (cx + r_loop*np.cos(theta_tip+1e-4), cy + r_loop*np.sin(theta_tip+1e-4))
    ax.add_patch(FancyArrowPatch(near, tip, arrowstyle='-|>', mutation_scale=head_size,
                                 lw=0, color=color))

def visualize_inverses(n):
    elems = zstar(n)
    k = len(elems)
    ang = np.linspace(0, 2*np.pi, k, endpoint=False)
    pos = {a: (np.cos(t), np.sin(t)) for a, t in zip(elems, ang)}
    invs = {a: inv_mod(a, n) for a in elems}

    fig, ax = plt.subplots(figsize=(7, 7))
    ax.set_aspect("equal"); ax.axis("off")

    # guide circle
    t = np.linspace(0, 2*np.pi, 400)
    ax.plot(np.cos(t), np.sin(t), color="lightgrey", lw=0.7, alpha=0.5)

    for a, (x, y) in pos.items():
        ax.plot(x, y, "o", ms=9, color="#8ecae6", zorder=2)
        ax.text(1.30*x, 1.30*y, str(a), ha="center", va="center", fontsize=12, zorder=3)

    drawn = set()
    for a in elems:
        b = invs[a]
        x1, y1 = pos[a]
        if a == b:
            draw_smooth_self_loop(ax, x1, y1)
            continue
        if (b, a) in drawn:
            continue
        drawn.add((a, b))
        x2, y2 = pos[b]
        ax.add_patch(FancyArrowPatch(
            (x1, y1), (x2, y2),
            arrowstyle="<->", mutation_scale=12,
            lw=1.6, color="#b22222"
        ))

    ax.set_xlim(-1.6, 1.6)
    ax.set_ylim(-1.6, 1.6)
    plt.show()

# ---------- UI ----------
top_label = widgets.HTML("<div style='font-size:16px'><b>Group:</b> Z*<sub>n</sub> &nbsp; (choose n)</div>")
n_text    = widgets.Text(value="13", description="n =", layout=widgets.Layout(width="240px"))
go_btn    = widgets.Button(description="Update", button_style="primary", layout=widgets.Layout(width="100px"))
head      = widgets.HBox([top_label, n_text, go_btn])
out       = widgets.Output()

def render(_=None):
    with out:
        clear_output(wait=True)
        try:
            n = int(n_text.value)
            if n < 3:
                display(Math(r"\text{Please choose } n\ge 3."))
                return
            elems = zstar(n)
            elems_tex = r",\, ".join(map(str, elems))

            # Clean LaTeX display
            display(Math(rf"\mathbb{{Z}}_{{{n}}}^{{\ast}}=\{{{elems_tex}\}}"))
            display(Math(rf"|\mathbb{{Z}}_{{{n}}}^{{\ast}}|=\varphi({n})={len(elems)}"))

            visualize_inverses(n)
        except Exception as e:
            display(Math(rf"\text{{Error: }} {str(e)}"))

go_btn.on_click(render)
display(head)
render()
display(out)


Output()

<details>
<summary><b>✅ Click to reveal the answer</b></summary>

<br>

**Proof.**

Let $a \in \mathbb{Z}_n^*$ and $a^{-1}$ be its inverse, so  

$$
a \cdot a^{-1} \equiv 1 \pmod{n}.
$$

Multiply both factors by $-1$:  

$$
(-a) \cdot (-a^{-1}) = a \cdot a^{-1} \equiv 1 \pmod{n}.
$$

Hence the inverse of $-a$ is $-a^{-1}$ (mod $n$).  
Since $-x \equiv n - x \pmod{n}$, we get  

$$
(n - a)(n - a^{-1}) \equiv 1 \pmod{n}.
$$

Therefore,  

$$
a \leftrightarrow a^{-1}
\;\Longrightarrow\;
(n - a) \leftrightarrow (n - a^{-1}),
$$

as desired.

</details>


### 📘 Definition 8.2.2 — Finite Group  

A group $ (G, \circ) $ is called **finite** if it has only finitely many elements.  
The number of elements in $ G $ is called the **cardinality** or **order** of the group and is written as $ |G| $.  

---

### 🔑 Intuition  
A finite group is just a group with a limited number of elements.  
Its size $ |G| $ is an important quantity, since many theorems (like Lagrange’s) directly relate subgroup sizes to $ |G| $.



### 📘 Definition 8.2.3 — Order of an Element  

Let $ (G,\circ) $ be a group and let $ a \in G $.  
The **order** of $ a $ is the smallest positive integer $ k $ such that  

$$
a^{k} = 1
$$  

(where $1$ is the identity element of $ G $).  

If no such $ k $ exists, then $ a $ has **infinite order**.  

---

🔑 **Intuition (concise):**  
The order of $a$ is the number of times the group operation must be applied to $a$ with itself until it becomes the identity element.




### 🧮 Example 8.5 — Order of an Element in $ \mathbb{Z}_{11}^* $

We try to determine the order of $a=3$ in the group $ \mathbb{Z}_{11}^* $.  
Compute successive powers until we reach the identity $1$:

$$
\begin{aligned}
a^1 &= 3,\\
a^2 &= a\cdot a = 3\cdot 3 = 9,\\
a^3 &= a^2\cdot a = 9\cdot 3 = 27 \equiv 5 \pmod{11},\\
a^4 &= a^3\cdot a = 5\cdot 3 = 15 \equiv 4 \pmod{11},\\
a^5 &= a^4\cdot a = 4\cdot 3 = 12 \equiv 1 \pmod{11}.
\end{aligned}
$$

From the last line we conclude $ \operatorname{ord}(3)=5 $.


### 📝 Quiz: Implement the Order Function

Implement a function `getOrder(a, n)` that returns the order of element $a$ in $\mathbb{Z}_n^*$.  
If $a \notin \mathbb{Z}_n^*$, return `"Not an element."`.

In [ ]:
def getOrder(a, n):
    g = Group(n)
    if a % n not in g.elements:
        return "Not an element."
    # *** Your code here ***
    # Hints:
    # - Use g.multiplicativeGroupOp(x, y) for the group operation
    # - The identity is 1
    # - Start from accumulator = 1 and keep multiplying by a (mod n)
    # - The order is the smallest k >= 1 such that accumulator == 1


<details>
<summary>✅ Click to reveal the solution</summary>

```python
def getOrder(a, n):
    g = Group(n)
    a = a % n
    if a not in g.elements:
        return "Not an element."
    k = 0
    acc = 1
    # Order must divide |G|; loop is safe up to |G|
    for _ in range(1, g.getCardinality() + 1):
        acc = g.multiplicativeGroupOp(acc, a)
        k += 1
        if acc == 1:
            return k
    # Should never happen for a in the group, but guard anyway
    return None

# Quick checks
print(getOrder(2, 11))  # 10  (generator in Z*_11)
print(getOrder(10, 11)) # 2
print(getOrder(4, 11))  # 5
print(getOrder(6, 11))  # 10
print(getOrder(0, 11))  # "Not an element."


### 🔒 How Large Should the Order Be for Security?  

In cryptography, the *order* of the chosen element (generator) must be extremely large to resist attacks:  

- For classical security:  
  - Groups of order around $2^{2048}$ (≈ 600 decimal digits) are recommended today.  
  - This corresponds to primes $p$ of about 2048 bits in Diffie–Hellman.  

- For elliptic curve cryptography (ECC):  
  - Much smaller groups (≈ $2^{256}$ elements) are enough, since the discrete log problem is harder on elliptic curves.  

- Why so large?  
  - The best known attacks on discrete logarithms run in **sub-exponential time** for $ \mathbb{Z}_p^* $.  
  - If the order is too small (say, $10^6$ or even $10^{12}$), attackers could brute-force or use index calculus methods in practical time.  

✅ **Takeaway:**  
For modern security, we need group elements whose order is astronomically large — typically hundreds of digits. This ensures that cycling back to the identity takes longer than the age of the universe with current computers.


### 📖 Definition 8.2.4 — Cyclic Group

A group $G$ which contains an element $\alpha$ with maximum order  

$$
\operatorname{ord}(\alpha) = |G|
$$

is said to be **cyclic**.  
Elements with maximum order are called **primitive elements** or **generators**.

---

#### 📘 Example 8.6: A Primitive Element in $\mathbb{Z}_{11}^*$

Consider the group:

$$
\mathbb{Z}_{11}^* = \{1,2,3,4,5,6,7,8,9,10\},
\quad |\mathbb{Z}_{11}^*| = 10
$$

Check whether $a = 2$ is a generator:

$$
\begin{aligned}
2^1 &\equiv 2 \pmod{11}, \\
2^2 &\equiv 4 \pmod{11}, \\
2^3 &\equiv 8 \pmod{11}, \\
2^4 &\equiv 5 \pmod{11}, \\
2^5 &\equiv 10 \pmod{11}, \\
2^6 &\equiv 9 \pmod{11}, \\
2^7 &\equiv 7 \pmod{11}, \\
2^8 &\equiv 3 \pmod{11}, \\
2^9 &\equiv 6 \pmod{11}, \\
2^{10} &\equiv 1 \pmod{11}.
\end{aligned}
$$

Since $\operatorname{ord}(2) = 10 = |\mathbb{Z}_{11}^*|$,  
the element $2$ is a **primitive element** and $\mathbb{Z}_{11}^*$ is **cyclic**, generated by $2$.


### 📝 Quiz: Implement the Cyclic Group Check

Implement a method `isCyclic(self)` for the `Group` class that checks whether  
$\mathbb{Z}_n^*$ is cyclic.  
The function should return `True` if there exists an element whose order equals  
the group’s cardinality, and `False` otherwise.  

In [ ]:
# Add/override this method for the Group class.
def isCyclic(self):
    # *** Your code here ***
    # Hints:
    # - Let m = self.getCardinality()
    # - For each a in self.elements, compute the order of a
    # - If any has order == m, return True; else return False
    raise NotImplementedError

# Bind stub to the class so students can run tests after implementing.
Group.isCyclic = isCyclic


<details>
<summary>✅ Click to reveal the solution</summary>

```python
def isCyclic(self):
    m = self.getCardinality()
    for a in self.elements:
        if getOrder(a, self.modulus) == m:
            return True
    return False

# Attach to Group
Group.isCyclic = isCyclic

# ✅ Test cases
print(Group(11).isCyclic())  # True (Z*_11 is cyclic)
print(Group(8).isCyclic())   # False (Z*_8 is not cyclic)
print(Group(9).isCyclic())   # True (Z*_9 is cyclic)


## 🔥 Wrapping Up: Why Group Theory Rules Modern Cryptography

1️⃣ **Fast Keys, Fast Encryption, Fast Decryption**  
We want cryptosystems like **ElGamal** or **Diffie–Hellman** to be lightning fast ⚡ —  that means generating keys quickly, encrypting quickly, and decrypting without melting your laptop.  
Group theory gives us a mathematical playground where all these operations behave nicely and predictably.

---

2️⃣ **💀 Beat the Hackers**  
Hackers hate math (well, most of them 😎).  
Our goal is to make the “reverse” problem — the **discrete logarithm** — so hard  
that even the fastest computer in the universe can’t crack it before the Sun burns out.  
That’s why we carefully choose **groups** where it’s easy to go forward (`g^x mod p`)  
but nearly impossible to go backward (find `x`).

---

3️⃣ **🎯 The Secret Sauce — Subgroups and Orders**  
Every cyclic group hides little “sub-worlds” called **subgroups**.  
Finding a *primitive element* (a generator) for the *whole* group might take effort,  
but sometimes we only need a **large prime-order subgroup** — still huge enough to defeat hackers, and much faster to compute with.  
That’s how ElGamal and Diffie–Hellman stay both *secure* and *efficient*.

---

4️⃣ **🌌 X25519 — The Curve that Protects the Internet**  
Modern browsers use an elliptic curve called **X25519** for key exchange.  
Like the number group $ \mathbb{Z}_p^* $, the points on **X25519** form a **finite cyclic group** — there’s a generator point, and every other point on the curve can be reached by repeatedly “adding” it to itself. Its group has about **2²⁵⁶ ≈ 1.16 × 10⁷⁷** possible elements 🤯  
Compare that to the number of atoms in the observable universe — roughly **10⁸⁰**.  
Basically, brute-forcing is physically impossible in practice.

---

💡 **Takeaway:**  
All this wizardry — from ElGamal to your browser’s HTTPS padlock 🔒 comes down to understanding the *rhythm of groups*: identity, inverses, orders, and generators, subgroups etc.
Mathematics isn’t just abstract — it’s literally what keeps our internet safe!

✨ *MATH = Making Algorithms Truly Hard (so hackers cry).*


In [ ]:
class CyclicGroup(Group):
    def getOrder(self, a):
        # Keep original contract
        if a not in self.elements:
            return "Not an element."
        a = a % self.modulus
        accumulator = 1
        k = 0
        # Lagrange: order(a) divides |G|, so loop ≤ |G|
        for _ in range(1, self.getCardinality() + 1):
            accumulator = self.multiplicativeGroupOp(accumulator, a)
            k += 1
            if accumulator == 1:
                return k
        # Shouldn't happen for valid elements; keep style (no exceptions)
        return None

    def isCyclic(self):
        for a in self.elements:
            if self.getOrder(a) == self.getCardinality():
                return True
        return False

    def findGenerators(self):
        generators = []
        for a in self.elements:
            if self.getOrder(a) == self.getCardinality():
                generators.append(a)
        return generators

    def generateSubgroups(self):
        results = set()
        generators = self.findGenerators()  # keep your original filtering
        for a in self.elements:
            if a not in generators:
                ord_a = self.getOrder(a)
                if isinstance(ord_a, int):
                    # Build the subgroup ⟨a⟩ = {a^0, a^1, ..., a^{ord(a)-1}}
                    sub = {pow(a, i, self.modulus) for i in range(ord_a)}
                    # Stable string for de-dup (sorted to avoid set order randomness)
                    results.add(str(sorted(sub)))
        return results

    def generateElements(self, generator):
        # Return exactly ord(generator) elements; include 1 = generator^0
        ord_g = self.getOrder(generator)
        if ord_g == "Not an element." or ord_g is None:
            return "Not an element."
        return {pow(generator, i, self.modulus) for i in range(ord_g)}
